In [3]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import errorcode
import time
from datetime import datetime
import re

# Database connection
conn = mysql.connector.connect(
   user= "wsa",
   host= "34.68.250.121",
   database = "ufcElo",
   password = "LeBron>MJ!"
)
cursor = conn.cursor()

In [4]:
# Function to insert events
def insert_event(name, date):
    cursor.execute("INSERT INTO events (name, date) VALUES (%s, %s)", (name, date))
    conn.commit()
    return cursor.lastrowid

In [21]:
def insert_fight(event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url):
    try:
        cursor.execute("""
            INSERT INTO fights (event_id, fighter1_id, fighter2_id, weight_class, result, method, rounds, time, outcome, url)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url))
        conn.commit()  # ✅ Explicitly commit after insert
    except mysql.connector.Error as err:
        print("Error inserting fight:", err)
        conn.rollback()  # 🚨 Rollback if there's an issue

In [6]:
def insert_fighter(fighter_name, weight_class):
    cursor.execute("SELECT fighter_id FROM fighters WHERE name = %s", (fighter_name,))
    result = cursor.fetchone()

    if result:
        return result[0]
    else:
        cursor.execute("""
            INSERT INTO fighters (name, weightclass, wins, losses, draws, currentElo, highestElo)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (fighter_name, weight_class, 0, 0, 0, 1500, 1500))  # Default Elo is 1500
        conn.commit()
        return cursor.lastrowid

In [7]:
# Function to update Elo ratings in the database
def update_elo(winner_id, loser_id, is_title_fight):
    # Fetch current Elo ratings from the database
    cursor.execute("SELECT currentElo FROM fighters WHERE fighter_id = %s", (winner_id,))
    winner_elo = cursor.fetchone()[0]
    
    cursor.execute("SELECT currentElo FROM fighters WHERE fighter_id = %s", (loser_id,))
    loser_elo = cursor.fetchone()[0]
    
    # Expected score function
    def expected_score(rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    # Elo constants
    k_factor = 50
    
    # Calculate expected scores
    expected_winner = expected_score(winner_elo, loser_elo)
    expected_loser = expected_score(loser_elo, winner_elo)
    
    # Update Elo based on the fight result
    if is_title_fight:
        winner_new_elo = winner_elo + (k_factor * (1 - expected_winner) * 2)
        loser_new_elo = loser_elo + k_factor * (0 - expected_loser)
    else:
        winner_new_elo = winner_elo + k_factor * (1 - expected_winner)
        loser_new_elo = loser_elo + k_factor * (0 - expected_loser)
    
    # Update current Elo for both fighters in the database
    cursor.execute("UPDATE fighters SET currentElo = %s WHERE fighter_id = %s", (winner_new_elo, winner_id))
    cursor.execute("UPDATE fighters SET currentElo = %s WHERE fighter_id = %s", (loser_new_elo, loser_id))

    # Update the highest Elo (all-time Elo) if necessary for winner
    cursor.execute("SELECT highestElo FROM fighters WHERE fighter_id = %s", (winner_id,))
    current_alltime_elo = cursor.fetchone()[0]
    if winner_new_elo > current_alltime_elo:
        cursor.execute("UPDATE fighters SET highestElo = %s WHERE fighter_id = %s", (winner_new_elo, winner_id))
    
    # Update the highest Elo (all-time Elo) if necessary for loser
    cursor.execute("SELECT highestElo FROM fighters WHERE fighter_id = %s", (loser_id,))
    current_alltime_elo = cursor.fetchone()[0]
    if loser_new_elo > current_alltime_elo:
        cursor.execute("UPDATE fighters SET highestElo = %s WHERE fighter_id = %s", (loser_new_elo, loser_id))

    conn.commit()

In [8]:
def store_fight_stats(fight_id, fighter_id, stats):
    # Example SQL insert statement
    query = f"""
    INSERT INTO advanced_stats (fight_id, fighter_id, kd, sig_str, sig_str_pct, total_str, td, td_pct, sub_att, rev, ctrl, head, body, leg, distance, clinch, ground)
    VALUES ({fight_id}, {fighter_id}, {stats['kd']}, {stats['sig_str']}, {stats['sig_str_pct']}, {stats['total_str']}, {stats['td']}, {stats['td_pct']}, 
            {stats['sub_att']}, {stats['rev']}, {stats['ctrl']}, {stats['head']}, {stats['body']}, {stats['leg']}, {stats['distance']}, {stats['clinch']}, {stats['ground']})
    """
    # Execute the query to store the stats in the database
    # (You would replace this with actual database execution code, like using cursor.execute(query))
    print("Storing stats:", query)

In [9]:
def get_fight_stats(fight_url, fight_id, fighter1_id, fighter2_id):
    url = fight_url
    headers = {"User-Agent": "Mozilla/5.0"}

    # Fetch the webpage
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all tables
    tables = soup.find_all("table")

    fight_stats_table = tables[0]  # Replace with the correct table index
    fight_stats_table2 = tables[2]  # Replace with the correct table index
    rows = fight_stats_table.find_all("tr")
    rows2 = fight_stats_table2.find_all("tr")
    fighter_stats = {
        'fighter1': {},
        'fighter2': {}
    }

    fighter_stats = {
        'fighter1': {'fighter_id': fighter1_id, 'stats': {}},
        'fighter2': {'fighter_id': fighter2_id, 'stats': {}}
    }

    # Process the stats from fight_stats_table
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the fighter names and stats
            fighter_names = [name.strip() for name in cols[0].text.split("\n") if name.strip()]
            if len(fighter_names) == 2:
                fighter1_name = fighter_names[0].strip()
                fighter2_name = fighter_names[1].strip()

                # Stats for fighter1
                stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
                fighter_stats['fighter1']['stats'] = {
                    'kd': stats[0].split()[0],  # Knockdowns
                    'sig_str': stats[1].split()[0],  # Significant Strikes
                    'sig_str_pct': stats[2].split()[0],  # Significant Strikes %
                    'total_str': stats[3].split()[0],  # Total Strikes
                    'td': stats[4].split()[0],  # Takedowns
                    'td_pct': stats[5].split()[0],  # Takedown %
                    'sub_att': stats[6].split()[0],  # Submission Attempts
                    'rev': stats[7].split()[0],  # Reversals
                    'ctrl': stats[8].split()[0],  # Control Time
                }

                # Stats for fighter2 (same as fighter1, but second stat)
                fighter_stats['fighter2']['stats'] = {
                    'kd': stats[0].split()[1],
                    'sig_str': stats[1].split()[3],
                    'sig_str_pct': stats[2].split()[1],
                    'total_str': stats[3].split()[3],
                    'td': stats[4].split()[3],
                    'td_pct': stats[5].split()[1],
                    'sub_att': stats[6].split()[1],
                    'rev': stats[7].split()[1],
                    'ctrl': stats[8].split()[1],
                }

    # Process advanced stats (from fight_stats_table2)
    for row in rows2:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the advanced stats for each fighter
            stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
            fighter_stats['fighter1']['stats'].update({
                'head': stats[2].split()[0],
                'body': stats[3].split()[0],
                'leg': stats[4].split()[0],
                'distance': stats[5].split()[0],
                'clinch': stats[6].split()[0],
                'ground': stats[7].split()[0]
            })

            fighter_stats['fighter2']['stats'].update({
                'head': stats[2].split()[3],
                'body': stats[3].split()[3],
                'leg': stats[4].split()[3],
                'distance': stats[5].split()[3],
                'clinch': stats[6].split()[3],
                'ground': stats[7].split()[3]
            })
    store_fight_stats(fight_id, fighter_stats['fighter1']['fighter_id'], fighter_stats['fighter1']['stats'])
    store_fight_stats(fight_id, fighter_stats['fighter2']['fighter_id'], fighter_stats['fighter2']['stats'])
    #print(fighter_stats)    store_fight_stats(fight_id, fighter_stats['fighter2']['fighter_id'], fighter_stats['fighter2']['stats'])
    #return fighter_stats
    #return fighter_stats
#get_fight_stats("http://www.ufcstats.com/fight-details/9779eb1aae1d4654")

In [10]:
def update_fighter_record(fighter_id, outcome):
    if outcome == "win":
        cursor.execute("UPDATE fighters SET wins = wins + 1 WHERE fighter_id = %s", (fighter_id,))
    elif outcome == "loss":
        cursor.execute("UPDATE fighters SET losses = losses + 1 WHERE fighter_id = %s", (fighter_id,))
    elif outcome == "draw":
        cursor.execute("UPDATE fighters SET draws = draws + 1 WHERE fighter_id = %s", (fighter_id,))
    
    conn.commit()

In [11]:
def get_fight_id_from_url(fight_url):

    # SQL query to find the fight_id based on the fight_url
    cursor.execute("SELECT fight_id FROM fights WHERE fight_url = ?", (fight_url,))
    
    # Fetch the result (it should return a single row with fight_id)
    result = cursor.fetchone()
    
    if result:
        return result[0]  # Return the fight_id
    else:
        return None  # Return None if the fight_url is not found
    

In [12]:
# Function to process each fight
def process_fights(fight_data):
    for fight in fight_data:
        try:
            # Extract basic fight information
            winner = fight[1]
            loser = fight[2]
            weight_class = fight[3]
            title_fight = fight[4] == "Yes"
            method = fight[5]
            round_end = fight[6]
            fight_time = fight[7]
            outcome = fight[8]
            url = fight[9]

            # Insert fighters into the database and get their IDs
            winner_id = insert_fighter(winner, weight_class)  # Insert or get fighter ID for winner
            loser_id = insert_fighter(loser, weight_class)  # Insert or get fighter ID for loser
            
            # Insert the fight into the database
            insert_fight(fight[0], winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time, outcome, url)

            # Fetch and store advanced stats for both fighters
            fight_url = fight[9]
            fightId = get_fight_id_from_url(fight_url)
            get_fight_stats(fight_url, fightId, winner_id, loser_id)
            

            # Update fighter records (wins, losses, draws)
            if outcome == "win":
                update_fighter_record(winner_id, "win")
                update_fighter_record(loser_id, "loss")
                update_elo(winner_id, loser_id, title_fight)  # Update Elo rating based on the outcome (only if win)
            elif outcome == "Draw":
                update_fighter_record(winner_id, "draw")
                update_fighter_record(loser_id, "draw")

        except Exception as e:
            print(f"Error processing fight {fight[0]}: {e}")

In [13]:
def get_fight_links(event_url):
    event_url_event = requests.get(event_url)
    event_soup = BeautifulSoup(event_url_event.text, "html.parser")
    
    fight_links = []
    fight_table = event_soup.find("table", class_="b-fight-details__table")
    
    if fight_table:
        for row in fight_table.find_all("tr")[1:]:  # Skip header row
            link = row.find("a", class_="b-link_style_black")
            if link and link.get("href"):
                fight_links.append("https://www.ufc.com" + link.get("href"))
    
    return fight_links

In [14]:
def get_fight_id(fighter_name):
    # Query to get fighter_id by name (adjust based on your database schema)
    cursor.execute("SELECT fighter_id FROM fighters WHERE fighter_name = ?", (fighter_name,))
    
    # Fetch the result
    result = cursor.fetchone()

    # If the fighter exists in the database, return their fighter_id
    if result:
        return result[0]  # Assuming fighter_id is the first column in the result
    else:
        # If the fighter is not found, return None or handle it differently
        print(f"Fighter '{fighter_name}' not found in the database.")
        return None

In [19]:
# Scrape fight data
fight_data = []

url = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all')
soup = BeautifulSoup(url.text, 'html.parser')
event_links = [link["href"] for link in soup.select(".b-statistics__table-row a")]
event_links.reverse()

for event_url in event_links:
    event_url_event = requests.get(event_url)
    event_soup = BeautifulSoup(event_url_event.text, "html.parser")
    
    # Get event details
    event_name = event_soup.select_one(".b-content__title").text.strip()
    event_date_raw = event_soup.select_one(".b-list__box-list-item:first-child").text.strip()
    event_date_clean = re.search(r"([A-Za-z]+ \d{1,2}, \d{4})", event_date_raw)
    event_date = datetime.strptime(event_date_clean.group(1), "%B %d, %Y").strftime("%Y-%m-%d") if event_date_clean else None
    
    # Insert event
    event_id = insert_event(event_name, event_date)

    
    fight_table = event_soup.find("table", class_="b-fight-details__table")
    
    if fight_table:
        for row in fight_table.find_all("tr")[1:]:  # Skip header row
            cols = [col.text.strip() for col in row.find_all("td")]
            if len(cols) > 6:
                fight_link = row.get('data-link')

                print(f"Fight URL: {fight_link}")  # Debugging print

                fighters = [fighter.strip() for fighter in cols[1].split("\n") if fighter.strip()]
                winner = fighters[0] if fighters else "Unknown"
                loser = fighters[-1] if len(fighters) > 1 else "Unknown"
                weight_class = cols[6]
                method = [m.strip() for m in cols[7].split("\n") if m.strip()]
                true_method = method[0] if method else "Unknown"
                round_end = cols[8] if len(cols) > 8 else "Unknown"
                rounds = int(round_end.split(" ")[0])
                time = cols[9] if len(cols) > 9 else "Unknown"
                title_fight = "Yes" if row.find("img", src=lambda x: x and "belt.png" in x) else "No"
                
                # Determine the outcome
                outcome = "win"
                if "nc" in cols[0]:
                    outcome = "nc"
                elif "draw" in cols[0]:
                    outcome = "draw"
                
                #print(f"Outcome: {outcome}")  # This will print the outcome for each fight
                print(f"Fight data: {event_id}, {winner}, {loser}, {weight_class}, {title_fight}, {true_method}, {round_end}, {time}, {outcome}, {fight_link}")  # Debugging print
                
                # Insert the fight data into the database
                fight_data.append([event_id, winner, loser, weight_class, title_fight, true_method, round_end, time, outcome, fight_link])

Fight URL: http://www.ufcstats.com/fight-details/00835554f95fa911
Fight data: 2726, Royce Gracie, Patrick Smith, Open Weight, Yes, KO/TKO, 1, 1:17, win, http://www.ufcstats.com/fight-details/00835554f95fa911
Fight URL: http://www.ufcstats.com/fight-details/aa161c7159741766
Fight data: 2726, Royce Gracie, Remco Pardoel, Open Weight, No, SUB, 1, 1:31, win, http://www.ufcstats.com/fight-details/aa161c7159741766
Fight URL: http://www.ufcstats.com/fight-details/8fbdde20b712b8da
Fight data: 2726, Patrick Smith, Johnny Rhodes, Open Weight, No, SUB, 1, 1:07, win, http://www.ufcstats.com/fight-details/8fbdde20b712b8da
Fight URL: http://www.ufcstats.com/fight-details/5655639baecd8f6a
Fight data: 2726, Royce Gracie, Jason DeLucia, Open Weight, No, SUB, 1, 1:07, win, http://www.ufcstats.com/fight-details/5655639baecd8f6a
Fight URL: http://www.ufcstats.com/fight-details/c4aa04781989699e
Fight data: 2726, Remco Pardoel, Orlando Wiet, Open Weight, No, KO/TKO, 1, 1:29, win, http://www.ufcstats.com/fig

In [ ]:
# Process fights and update Elo ratings
process_fights(fight_data)

# Close connection
cursor.close()
conn.close()